In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd 

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [14]:
df = pd.read_csv("train_set.csv")
df

,label,id_img,path
0,happy,22373,happy/22373.jpg
1,happy,21433,happy/21433.jpg
2,happy,12418,happy/12418.jpg
3,happy,21278,happy/21278.jpg
4,happy,8081,happy/08081.jpg
...,...,...,...
6171,sadness,11346,sadness/11346.jpg
6172,sadness,4441,sadness/04441.jpg
6173,sadness,15236,sadness/15236.jpg
6174,sadness,27361,sadness/27361.jpg


In [15]:
from pycaret.classification import *

setup = setup(data=df, target="label", numeric_imputation="median", session_id=123)


,Description,Value
0,session_id,123
1,Target,label
2,Target Type,Binary
3,Label Encoded,"happy: 0, sadness: 1"
4,Original Data,"(6176, 3)"
5,Missing Values,False
6,Numeric Features,1
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [16]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.6343,0.6184,0.2524,0.9088,0.3947,0.2380,0.3389,1.7700
ada,Ada Boost Classifier,0.6334,0.6327,0.2422,0.9329,0.3843,0.2353,0.3444,0.6170
catboost,CatBoost Classifier,0.6331,0.6191,0.2456,0.9222,0.3873,0.2351,0.3407,2.5860
gbc,Gradient Boosting Classifier,0.6329,0.6261,0.2417,0.9331,0.3836,0.2344,0.3436,2.0960
et,Extra Trees Classifier,0.6267,0.6258,0.2837,0.8001,0.4177,0.2254,0.2921,3.2240
lightgbm,Light Gradient Boosting Machine,0.6132,0.6163,0.3669,0.6679,0.4724,0.2058,0.2298,0.1530
nb,Naive Bayes,0.6123,0.6283,0.4482,0.6270,0.5225,0.2108,0.2196,0.1340
ridge,Ridge Classifier,0.6063,0.0000,0.5069,0.6145,0.5420,0.2039,0.2129,5.5320
xgboost,Extreme Gradient Boosting,0.6003,0.6153,0.4183,0.6135,0.4970,0.1848,0.1943,7.7780
dt,Decision Tree Classifier,0.5950,0.5872,0.4433,0.5979,0.5079,0.1767,0.1830,0.1770


In [17]:
print(best_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=123, verbose=0,
                       warm_start=False)


In [18]:
rf = create_model("rf")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6374,0.6392,0.2488,0.9444,0.3938,0.2447,0.3561
1,0.6467,0.6553,0.2732,0.9333,0.4226,0.2651,0.3694
2,0.6166,0.5918,0.2390,0.8305,0.3712,0.2024,0.2840
3,0.6204,0.6138,0.2157,0.9167,0.3492,0.2065,0.3148
4,0.6181,0.6204,0.2500,0.8095,0.3820,0.2048,0.2792
5,0.6528,0.6288,0.2892,0.9219,0.4403,0.2773,0.3756
6,0.6296,0.5640,0.2353,0.9231,0.3750,0.2266,0.3341
7,0.6481,0.6381,0.2598,0.9815,0.4109,0.2657,0.3856
8,0.6204,0.6139,0.2402,0.8448,0.3740,0.2086,0.2939
9,0.6528,0.6186,0.2732,0.9825,0.4275,0.2785,0.3966


In [20]:
tuned_rf = tune_model(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6467,0.6278,0.2732,0.9333,0.4226,0.2651,0.3694
1,0.6490,0.6303,0.2780,0.9344,0.4286,0.2701,0.3739
2,0.6166,0.5937,0.2390,0.8305,0.3712,0.2024,0.2840
3,0.6181,0.5969,0.2157,0.8980,0.3478,0.2018,0.3050
4,0.6181,0.5987,0.2500,0.8095,0.3820,0.2048,0.2792
5,0.6481,0.6293,0.2892,0.8939,0.4370,0.2681,0.3587
6,0.6343,0.6138,0.2451,0.9259,0.3876,0.2367,0.3435
7,0.6481,0.6280,0.2647,0.9643,0.4154,0.2661,0.3804
8,0.6227,0.6028,0.2451,0.8475,0.3802,0.2136,0.2989
9,0.6528,0.6346,0.2780,0.9661,0.4318,0.2789,0.3915


In [21]:
tuned_rf

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='gini',
                       max_depth=4, max_features='sqrt', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0005,
                       min_impurity_split=None, min_samples_leaf=3,
                       min_samples_split=5, min_weight_fraction_leaf=0.0,
                       n_estimators=260, n_jobs=-1, oob_score=False,
                       random_state=123, verbose=0, warm_start=False)

In [22]:
data_unseen = pd.read_csv('test_set.csv') 
# generate predictions on unseen data
predictions = predict_model(tuned_rf, data = data_unseen)
predictions

,id_img,path,Label,Score
0,18341,test/18341.jpg,happy,0.5
1,13176,test/13176.jpg,happy,0.5
2,23945,test/23945.jpg,happy,0.5
3,15968,test/15968.jpg,happy,0.5
4,18382,test/18382.jpg,happy,0.5
...,...,...,...,...
4112,8966,test/08966.jpg,happy,0.5
4113,12111,test/12111.jpg,happy,0.5
4114,16629,test/16629.jpg,happy,0.5
4115,24322,test/24322.jpg,happy,0.5


In [43]:
submission = predictions.iloc[:,::2]

In [44]:
submission.rename(columns={"Label":"label"}, inplace=True)

In [45]:
sample = pd.read_csv("sample_submission.csv")
sample

,id_img,label
0,18341,sadness
1,13176,sadness
2,23945,happy
3,15968,happy
4,18382,happy
...,...,...
4112,8966,sadness
4113,12111,sadness
4114,16629,sadness
4115,24322,sadness


In [46]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id_img.all() == sample.id_img.all():
                print("You're ready to submit!")
                submission.to_csv("submission_pycaret.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")

In [47]:
chequeator(submission)

You're ready to submit!


NameError: name 'urllib' is not defined